In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import scipy

In [2]:
#!pip install scikit-learn

In [2]:
df = pd.read_csv('train.csv')
df.shape

(750156, 4)

In [3]:
df = df[:100000]
df.shape

(100000, 4)

In [4]:
def rating(x):
    if x >=3:
        return 1
    else:
        return 0

In [5]:
df['rating'] = df['rating'].apply(rating)

In [7]:
#train_df = train_df.reset_index().iloc[:,1:]
#val_df = val_df.reset_index().iloc[:,1:]

In [8]:
#train_df.sort_values(by='user',inplace=True)
#val_df.sort_values(by='user',inplace=True)

In [6]:
user_df =  pd.get_dummies(df['user'],prefix='user').astype(int)
movie_df =  pd.get_dummies(df['movie'],prefix='movie').astype(int)

In [16]:
#columns= [['rating_'+str(i) for i in range(1,6041)]]
#indexs = movie_df.index
#data = np.zeros([600124,6040])
#ratings = pd.DataFrame(columns=columns,data=data)
#ratings.index = indexs
#ratings.head()

In [7]:
X = user_df.merge(movie_df,left_index=True,right_index=True)
Y = df['rating']

In [8]:
Y

0        1
1        1
2        1
3        1
4        1
        ..
99995    0
99996    0
99997    1
99998    1
99999    0
Name: rating, Length: 100000, dtype: int64

In [19]:
#X_train_sparse = scipy.sparse.csr_matrix(X_train.values)

In [20]:
#X_test = train_df['rating']
#X_test

In [21]:
#print(X_train.shape)
#print(X_test.shape)

In [22]:
#user_df =  pd.get_dummies(val_df['user'],prefix='user').astype(int)

In [23]:
#movie_df =  pd.get_dummies(val_df['movie'],prefix='movie').astype(int)

In [24]:
#y_train = user_df.merge(movie_df,left_index=True,right_index=True)

In [25]:
#y_test = val_df['rating']
#y_test

In [9]:
n = X.shape[0]
p = X.shape[1]
k = 10
batch_size = 128
epochs = 30

In [10]:
test = pd.read_csv('test.csv')
test.head()

,ID,user,movie
0,895537,5412,2683
1,899740,5440,904
2,55688,368,3717
3,63728,425,1721
4,822012,4942,3697


In [11]:
user_df =  pd.get_dummies(test['user'],prefix='user').astype(int)
movie_df =  pd.get_dummies(test['movie'],prefix='movie').astype(int)

In [12]:
X_test = user_df.merge(movie_df,left_index=True,right_index=True)

In [14]:
import tensorflow as tf

class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()

        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = tf.math.sigmoid(self.w_0 + linear_terms + interactions)

        return y_hat

In [19]:
# Forward
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False,
                                                   y_true=targets,
                                                   y_pred=y_pred)
    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy: update할 때마다 정확도는 누적되어 계산된다.
    accuracy.update_state(targets, y_pred)

    return loss


# 이전의 train 함수에서 호출 부분을 수정하여 모델을 평가하도록 변경합니다.
# 반복 학습 함수
def train(epochs):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(8)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(8)

    model = FM()
    optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
    accuracy = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
    loss_history = []

    for i in range(epochs):
      for x, y in train_ds:
          loss = train_on_batch(model, optimizer, accuracy, x, y)
          loss_history.append(loss)

      if i % 2== 0:
          print("스텝 {:03d}에서 누적 평균 손실: {:.4f}".format(i, np.mean(loss_history)))
          print("스텝 {:03d}에서 누적 정확도: {:.4f}".format(i, accuracy.result().numpy()))


    test_accuracy = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
    for x, y in test_ds:
        y_pred = model(x)
        test_accuracy.update_state(y, y_pred)

    print("테스트 정확도: {:.4f}".format(test_accuracy.result().numpy()))
    print('\n')
    print('평가 시작\n')
    rating =[]
    eval = tf.data.Dataset.from_tensor_slices(tf.cast(X_test, tf.float32)).shuffle(200).batch(8)
    for x in eval:
        try:
            y_pred = model(x)
            rating.append(y_pred)
        except:
            rating.append(1)

    return rating

In [20]:
fm = train(10)

스텝 000에서 누적 평균 손실: 0.9286
스텝 000에서 누적 정확도: 0.7270
스텝 002에서 누적 평균 손실: 0.8972
스텝 002에서 누적 정확도: 0.7444
스텝 004에서 누적 평균 손실: 0.8810
스텝 004에서 누적 정확도: 0.7490
스텝 006에서 누적 평균 손실: 0.8674
스텝 006에서 누적 정확도: 0.7518
스텝 008에서 누적 평균 손실: 0.8549
스텝 008에서 누적 정확도: 0.7539
테스트 정확도: 0.7545


평가 시작



eval 부분에서 평가를 진행하지 못함..ㅠ

In [33]:
len(fm)

2500

In [23]:
test_accuracy = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
test_accuracy.update_state([0.91065073, 0.87210613, 0.45799118, 0.99571574, 0.12938377,
        0.7050097 , 0.9852476 , 0.2945354 ], [0.91065073, 0.87210613, 0.45799118, 0.99571574, 0.12938377,
        0.7050097 , 0.9852476 , 0.2945354 ])

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=8.0>

# 함수

In [ ]:
def log_loss(pred, y):
    return np.log(np.exp(-pred * y) + 1.0)

In [ ]:
# Update gradients
def sgd(X, y, n_samples, n_features,
                w0, w, v, n_factors, learning_rate, reg_w, reg_v):
    data = X.data
    indptr = X.indptr
    indices = X.indices
    loss = 0.0

    for i in range(n_samples):
        pred, summed = predict(X, w0, w, v, n_factors, i)
        
        # calculate loss and its gradient
        loss += log_loss(pred, y[i])
        loss_gradient = -y[i] / (np.exp(y[i] * pred) + 1.0)
    
        # update bias/intercept term
        w0 -= learning_rate * loss_gradient 

        # update weight
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            w[feature] -= learning_rate * (loss_gradient * data[index] + 2 * reg_w * w[feature])

        # update factor
        for factor in range(n_factors):
            for index in range(indptr[i], indptr[i + 1]):
                feature = indices[index]
                term = summed[factor] - v[factor, feature] * data[index]
                v_gradient = loss_gradient * data[index] * term
                v[factor, feature] -= learning_rate * (v_gradient + 2 * reg_v * v[factor, feature])
    
    loss /= n_samples
    return loss

In [ ]:
def predict(X, w0, w, v, n_factors, i):
    data = X.data
    indptr = X.indptr
    indices = X.indices
    """predicting a single instance"""
    summed = np.zeros(n_factors)
    summed_squared = np.zeros(n_factors)

    # linear output w * x
    pred = w0
    for index in range(indptr[i], indptr[i + 1]):
        feature = indices[index]
        pred += w[feature] * data[index]

    # factor output
    for factor in range(n_factors):
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            term = v[factor, feature] * data[index]
            summed[factor] += term
            summed_squared[factor] += term * term
            
        
        pred += 0.5 * (summed[factor] * summed[factor] - summed_squared[factor])

    # gradient update할 때, summed는 독립이므로 re-use 가능
    return pred, summed

In [ ]:
# Train Factorization Machine
# X -> sparse csr_matrix, y -> label
def fit(X, y, config):
    epochs = config['num_epochs']
    num_factors = config['num_factors']
    learning_rate = config['learning_rate']
    reg_weights = config['reg_weights']
    reg_features = config['reg_features']

    num_samples, num_features = X.shape
    weights = np.zeros(num_features) # -> w
    global_bias = 0.0 # -> w0
    
    # latent factors for all features -> v
    feature_factors = np.random.normal(size = (num_factors, num_features))

    epoch_loss = []
    for epoch in range(epochs):
        
        #loss 값이 계속 떨어지면서 학습이 되는지 확인
        loss = sgd(X, y, num_samples, num_features,
                            global_bias, weights,
                            feature_factors, num_factors,
                            learning_rate, reg_weights, reg_features)
        print(f'[epoch: {epoch+1}], loss: {loss}')

        epoch_loss.append(loss)
      
    return epoch_loss

In [ ]:
#config = {
#    "num_epochs": 10,
#    "num_factors": 10,
#    "learning_rate": 0.1,
#    "reg_weights": 0.01,
#    "reg_features": 0.01
#}

In [ ]:
#epoch_loss = fit(X_train_sparse, y_train.values, config)

In [ ]:
#import matplotlib.pyplot as plt
#plt.plot(epoch_loss)
#plt.title('Loss per epoch')
#plt.show()